# Forward-backward splitting for time-varying graphical lasso
This notebook shows how to minimise the time-varying graphical lasso with element-wise penalty norms across time-points.

First of all, as always, let's create a bunch of data.
For this task, we generate eah variable to change according to a certain behaviour which can be described as evolution via tigonometric functions, such as `sin` and `cos`.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import squareform
from regain import datasets, utils

from sklearn.datasets import load_iris
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ShuffleSplit

from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [2]:
# np.random.seed(7)

# fs = 10e3
# N = 100
# amp = 2*np.sqrt(2)
# freq = 1.0
# noise_power = 0.001 * fs / 2
# time = np.arange(N) / fs
# z = amp*np.sin(2*np.pi*freq*time)
# z += np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
# plt.plot(z);

In [3]:
# T = 4

# x = np.tile(np.linspace(0, T-1, T), (n_interactions, 1))
# zz = amp * signal.square(2 * np.pi * freq * x + phase, duty=.5)
# plt.plot(x.T, zz.T);

Generate the data starting from the inverse covariance matrices.

In [4]:
np.random.seed(7)

# square
n_samples = 100
n_dim_obs = 20
T = 10

data = {}
data['square'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='square', closeness=2.4, normalize=1)

# smooth
n_samples = 100
n_dim_obs = 20
T = 10

data['smooth'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='smooth', closeness=2.4, normalize=1)

# plt.step(np.array([squareform(y, checks=None) for y in data.thetas]), '-|');
# plt.savefig("/home/fede/Dropbox/Latent variables networks/forward backward time varying graphical lasso/smooth_signal.pdf")

### Let's run 

In [5]:
# X = data.data
# X_tr, X_ts = train_test_split(X)

In [221]:
import pandas as pd
from functools import partial
from regain import update_rules; reload(update_rules);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from sklearn.covariance import GraphLasso, GraphLassoCV
import sys; sys.path.append("/home/fede/src/TVGL")
import inferGraphL1; reload(inferGraphL1)
import TVGL; reload(TVGL)

# use:
# beta = 2.1, norm = 1
# beta = 5.05, norm = 2
# prepare dataframe for results

methods = ['TGL-FB ($\ell_{21}$)', 'TGL-FB ($\ell_1$)', 'GL', 'TVGL ($\ell_{21}$)', 'TVGL ($\ell_1$)']

scores = sorted(['iter', 'accuracy',  'average_precision',  'balanced_accuracy',  'dor',  'f1',  'fall_out',  'false_omission_rate',  'fdr',  'fn',  'fp',  'miss_rate',  'nlr',  'npv',  'plr',  'precision',  'prevalence',  'recall',  'specificity',  'tn',  'tp'])
evolution = sorted(['square', 'smooth'])

rows = methods
cols = pd.MultiIndex.from_product([evolution, scores], names=('evolution', 'score'))
# rows = pd.MultiIndex.from_product([methods, n_times], names=('method','time'))

dff = pd.DataFrame(columns=cols, index=rows)
idx = pd.IndexSlice

In [237]:
def run_performance(beta=1, evolution='square'):
    X = data[evolution].data
    
    error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=0, eps=1e-5)
    
    tglfb = time_graph_lasso_.TimeGraphLassoForwardBackward(
        verbose=0, gamma=1, alpha=1.5, beta=beta,
        delta=.0001, choose='lamda',
        lamda=1, tol=1e-5, eps=0.7,
        time_norm=2, max_iter=300, time_on_axis='last').fit(X)
#     print(tglfb.alpha_max(X))
#     return
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    tglfb = tglfb.set_params(alpha=1.2, time_norm=1).fit(X)
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

    
    # gl = GraphLasso(alpha=tglfb.alpha/1.2)
    gl = GraphLassoCV(alphas=100)
    precision_gl = np.array([gl.fit(x).precision_.copy() for x in X.transpose(2,0,1)])
    res = error_function(precision_gl)
    res['iter'] = gl.n_iter_
    dff.loc['GL', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=tglfb.alpha, beta=tglfb.beta, indexOfPenalty=2, verbose=False)

    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=tglfb.alpha, beta=tglfb.beta, indexOfPenalty=1, verbose=False)
    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

In [334]:
evolution = 'square'
X = data[evolution].data
reload(time_graph_lasso_)
tglfb = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=2, gamma=1, alpha=1.2, beta=2.1,
    delta=.00001, choose='lamda', lamda_criterion='c',
    lamda=1, tol=1e-5, eps=0.7,
    time_norm=1, max_iter=300, time_on_axis='last').fit(X)

obj: 19594.2073, rnorm: 0.8285631, snorm: 99.5821,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19639.3980, rnorm: 0.5389933, snorm: 45.1906,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19619.5707, rnorm: 0.6219477, snorm: 19.8273,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19587.2615, rnorm: 0.6087891, snorm: 32.3091,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19606.4070, rnorm: 0.5542832, snorm: 19.1455,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19563.0715, rnorm: 0.5407609, snorm: 43.3355,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19563.5683, rnorm: 0.3953574, snorm: 0.4968,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19548.9310, rnorm: 0.3922214, snorm: 14.6373,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19544.3264, rnorm: 0.3402495, snorm: 4.6046,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19532.8852, rnorm: 0.3228679, snorm: 11.4411,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19538.8478, rnorm: 0.3613266, snorm: 5.9626,eps_pri: 0.0005, eps_dual: 0.0000
obj: 19519.1613, rnorm: 0.3532623, snorm: 19.6865,eps_pri: 0.0005, eps_dual: 0.

In [332]:
from regain.utils import positive_definite
positive_definite(tglfb.precision_)

True

In [335]:
tglfb.n_iter_

103

In [336]:
error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=1, eps=1e-3)

res = error_function(tglfb.precision_)
res

{'accuracy': 0.5925,
 'average_precision': 0.7738527204100978,
 'balanced_accuracy': 0.5047636315174506,
 'dor': 2.899876390605686,
 'f1': 0.7421701993040177,
 'fall_out': 0.9853836784409257,
 'false_omission_rate': 0.3333333333333333,
 'fdr': 0.40817356205852673,
 'fn': 12,
 'fp': 1618,
 'miss_rate': 0.005089058524173028,
 'nlr': 0.3481764206955047,
 'npv': 0.6666666666666666,
 'plr': 1.009668582140487,
 'precision': 0.5918264379414733,
 'prevalence': 0.5895,
 'recall': 0.9949109414758269,
 'specificity': 0.014616321559074299,
 'tn': 24,
 'tp': 2346}

In [302]:
error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=1, eps=1e-3)

res = error_function(tglfb.precision_)
res

{'accuracy': 0.6085,
 'average_precision': 0.7884074807058622,
 'balanced_accuracy': 0.5383063745468558,
 'dor': 2.2901082077867856,
 'f1': 0.7369835404769902,
 'fall_out': 0.853836784409257,
 'false_omission_rate': 0.40594059405940597,
 'fdr': 0.38987764182424917,
 'fn': 164,
 'fp': 1402,
 'miss_rate': 0.06955046649703138,
 'nlr': 0.47584110828385634,
 'npv': 0.594059405940594,
 'plr': 1.08972762768322,
 'precision': 0.6101223581757509,
 'prevalence': 0.5895,
 'recall': 0.9304495335029687,
 'specificity': 0.146163215590743,
 'tn': 240,
 'tp': 2194}

In [238]:
run_performance(beta=2.1, evolution='square')
# run_performance(beta=5.05, evolution='smooth')
run_performance(beta=2.1, evolution='smooth')
dff

KeyboardInterrupt: 

In [232]:
dff['smooth'].sort_values("f1", ascending=False)['iter']
# dff['square'].sort_values("f1", ascending=False)

TGL-FB ($\ell_{21}$)    81
TGL-FB ($\ell_1$)       20
GL                       3
TVGL ($\ell_{21}$)      21
TVGL ($\ell_1$)         27
Name: iter, dtype: int64

### BayesOptimisation
Since we have lots of hyper-parameters, we rely on a Bayesian optimisation procedure in order to select the best hyper-parameters, treating the scoring function of our algorithm as a black-box for the gaussian process underlying the Bayesian optimisation.

Such procedure is performed via the `scikit-optimize` package.

In [337]:
from regain import utils; reload(utils)
from regain import prox; reload(prox);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from skopt import searchcv; reload(searchcv)

X = data['square'].data

domain = {'alpha': Real(1e-1, 3, prior='uniform'),
          'beta': Real(1e-1, 1e1, prior='uniform'),
#           'time_norm': Categorical([1, 2])
#           'eps': Categorical([0.5, 0.7, 0.8, 0.9])
         }

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, tol=1e-2, max_iter=200, gamma=1, alpha='max', beta=2.1, time_norm=1,
    time_on_axis='last', eps=0.9, delta=1e-5, choose='lamda', lamda_criterion='c')
    
cv = ShuffleSplit(5, test_size=0.1)
    
bscv = searchcv.BayesSearchCV(
    mdl, domain, n_iter=20, cv=cv, verbose=1, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

def on_step(optim_result):
    score = bscv.best_score_
    print("best score: %s" % score)

bscv.fit(X, callback=on_step)
# mdl.fit(data_grid)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.9s finished


best score: -2951.806990371971
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   11.9s finished


best score: -2951.806990371971
Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    8.3s finished


best score: -2924.204655420389
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.7s finished


best score: -2924.204655420389


BayesSearchCV(cv=ShuffleSplit(n_splits=5, random_state=None, test_size=0.1, train_size=None),
       error_score=-300000.0,
       estimator=TimeGraphLassoForwardBackward(alpha='max', assume_centered=False, beta=2.1,
               choose='lamda', compute_objective=True, delta=1e-05,
               eps=0.9, gamma=1, lamda=1, lamda_criterion='c',
               max_iter=200, time_norm=1, time_on_axis='last', tol=0.01,
               verbose=0),
       fit_params=None, iid=True, n_iter=20, n_jobs=1, n_points=3,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'alpha': Real(low=0.1, high=3, prior='uniform', transform='identity'), 'beta': Real(low=0.1, high=10.0, prior='uniform', transform='identity')},
       verbose=1)

In [93]:
from regain import utils; reload(utils)
from regain import prox; reload(prox);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from skopt import searchcv; reload(searchcv)

X = data['smooth'].data

domain = {#'alpha': Real(1e-1, 1, prior='uniform'),
          'beta': Real(1e-1, 1e1, prior='log-uniform'),
#           'time_norm': Categorical([1, 2])
#           'eps': Categorical([0.5, 0.7, 0.8, 0.9])
         }

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, tol=1e-2, max_iter=200, gamma=1, alpha='max', beta=1, time_norm=2,
    time_on_axis='last', eps=0.9, delta=1e-5, choose='gamma')
    
cv = ShuffleSplit(5, test_size=0.1)
    
bscv_smooth = searchcv.BayesSearchCV(
    mdl, domain, n_iter=20, cv=cv, verbose=1, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

# callback handler
def on_step(optim_result):
    score = bscv_smooth.best_score_
    print("best score: %s" % score)
#     if score >= 0.98:
#         print('Interrupting!')
#         return True

bscv_smooth.fit(X, callback=on_step)
# mdl.fit(data_grid)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.2min finished


best score: -3045.4636277589393
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.8min finished


best score: -3009.3462324672396
Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.2min finished


best score: -2952.7948977072147
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.3min finished


best score: -2952.7948977072147


BayesSearchCV(cv=ShuffleSplit(n_splits=5, random_state=None, test_size=0.1, train_size=None),
       error_score=-300000.0,
       estimator=TimeGraphLassoForwardBackward(alpha='max', assume_centered=False, beta=1,
               choose='gamma', compute_objective=True, delta=1e-05,
               eps=0.9, gamma=1, lamda=1, lamda_criterion='b',
               max_iter=200, time_norm=2, time_on_axis='last', tol=0.01,
               verbose=0),
       fit_params=None, iid=True, n_iter=200, n_jobs=1, n_points=3,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'beta': Real(low=0.1, high=10.0, prior='log-uniform', transform='identity')},
       verbose=1)

In [338]:
bscv.best_params_

{'alpha': 3.0, 'beta': 4.986988949178272}

In [34]:
mdl = bscv.best_estimator_

In [32]:
utils.structure_error(data.thetas, mdl.precision_, no_diagonal=0, thresholding=1, eps=1e-8)

{'accuracy': 0.648,
 'average_precision': 0.945383085898071,
 'balanced_accuracy': 0.5616666666666666,
 'dor': 22.26436781609195,
 'f1': 0.772020725388601,
 'fall_out': 0.87,
 'false_omission_rate': 0.07142857142857142,
 'fdr': 0.3686440677966102,
 'fn': 4,
 'fp': 348,
 'miss_rate': 0.006666666666666667,
 'nlr': 0.05128205128205128,
 'npv': 0.9285714285714286,
 'plr': 1.1417624521072796,
 'precision': 0.6313559322033898,
 'prevalence': 0.6,
 'recall': 0.9933333333333333,
 'specificity': 0.13,
 'tn': 52,
 'tp': 596}

In [17]:
mdl.score(X)

-35.837641703814484

In [18]:
mdl.precision_

array([[[ 1.10743819,  0.        ,  0.68071616, -0.05748638,
          0.94732079,  0.        ,  0.20918433,  0.        ,
          1.78611225,  1.85242287],
        [ 0.        ,  1.07193912, -0.50367029,  1.92016201,
          0.21529756,  1.59502318,  2.31934029,  0.42497577,
          0.31269722,  1.75750906],
        [ 0.68071616, -0.50367029,  1.06188303,  1.68245203,
         -0.72654564,  3.31609044,  0.70047032,  1.23391116,
          0.55243893,  1.10080506],
        [-0.05748638,  1.92016201,  1.68245203,  1.13201969,
          4.03359096,  0.66731923, -0.58454924,  0.        ,
          1.06881747,  0.        ],
        [ 0.94732079,  0.21529756, -0.72654564,  4.03359096,
          1.16540114,  0.77208652,  1.10715018,  0.        ,
         -0.14574355,  0.2445326 ],
        [ 0.        ,  1.59502318,  3.31609044,  0.66731923,
          0.77208652,  1.0764689 ,  0.20839135, -1.00066514,
          1.02100579,  1.43229819],
        [ 0.20918433,  2.31934029,  0.70047032, -0.5

### GridSearchCV
As for the hyper-parameters tuning, one may choose to fix a grid of parameters and select the best ones.
For this we can use `GridSearchCV`, from the `scikit-learn` library.

In [ ]:
# data_grid = np.array(data.data).transpose(1,2,0)
param_grid=dict(alpha=np.logspace(-2,0,3), beta=np.logspace(-2,0,3), gamma=np.logspace(-2, 0, 3),
               time_norm=[1, 2])

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, time_on_axis='last')
    
cv = ShuffleSplit(2, test_size=0.2)
ltgl = GridSearchCV(mdl, param_grid, cv=cv, verbose=1)
ltgl.fit(data_grid)